In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import html
import re
import string
from fncbaseline.utils import dataset, generate_test_splits, score
from nltk.corpus import stopwords
from gensim.summarization import summarize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from fncbaseline.utils import dataset, generate_test_splits, score
from fncbaseline import feature_engineering
from nltk.corpus import stopwords
from gensim.summarization import summarize

from nltk.corpus import stopwords
import html
import re
from nltk import sent_tokenize
import nltk.data
from nltk.tokenize import word_tokenize

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')


/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Load the dataset
train_dataset = dataset.DataSet()
test_dataset = dataset.DataSet('test')

Reading dataset
Total stances: 49972
Total bodies: 1683
Reading dataset
Total stances: 25413
Total bodies: 904


In [3]:
def preprocess(text):
    text = html.unescape(text)
    text = text.replace("\\n"," ")
    #text = text.replace("\"","")
    text = text.replace("_NEG","")
    text = text.replace("_NEGFIRST", "")
    #text = re.sub(r"#", "", text)
    text = re.sub(r"\*", "", text)
    #text = re.sub(r"\'s", "", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    #text = re.sub(r",", "", text)
    #text = re.sub(r"!", " !", text)
    #text = re.sub(r"\(", "", text)
    #text = re.sub(r"\)", "", text)
    #text = re.sub(r"\?", " ?", text)
    #text = re.sub(r'[^\x00-\x7F]',' ', text)
    #text = re.sub(r'[^\w\s]',' ',text)
    #text = re.sub("\d+", " ", text)
    text = re.sub(r"\s{2,}", " ", text)
    #text = text.rstrip(',|.|;|:|\'|\"')
    #text = text.strip('\'|\"|" "')
    #text=text.lstrip('" "')
    text=text.replace('..', '. ')
    text=text.replace('--', '. ')
    text = text.replace('\n','. ')

    return text

In [4]:
data=dict()
for s in test_dataset.articles:
    a=list()
    text=test_dataset.articles[s]
    
    text=text.replace('”', '')
    text=text.replace('“', '')
    text=text.replace('"', '')
    text=text.replace('..', '. ')
    text=text.replace('--', '. ')
    text = text.replace('\n','. ')
    
    text=preprocess(text)
    sents=tokenizer.tokenize(text)
    if(len(sents)==1):
        sents=sents[0].split('.')
        
    for st in sents:
        st=preprocess(st)
        newsents=tokenizer.tokenize(st)
        for nst in newsents:
            if(len(nst)>1):
                splits=nst.split(' ')
                if(len(splits)>85):
                    a.append(' '.join(splits[:int(len(splits)/2)]))
                    a.append(' '.join(splits[int(len(splits)/2):]))
                    
                else:
                    a.append(nst)
            
    data[s]=(a)    

for s in train_dataset.articles:
    a=list()
    text=train_dataset.articles[s]
    
    text=text.replace('”', '')
    text=text.replace('“', '')
    text=text.replace('"', '')
    text=text.replace('..', '. ')
    text=text.replace('--', '. ')
    text = text.replace('\n','. ')
    
    text=preprocess(text)
    sents=tokenizer.tokenize(text)
    if(len(sents)==1):
        sents=sents[0].split('.')
    for st in sents:
        st=preprocess(st)
        newsents=tokenizer.tokenize(st)
        for nst in newsents:
            if(len(nst)>1):
                splits=nst.split(' ')
                if(len(splits)>85):
                    a.append(' '.join(splits[:int(len(splits)/2)]))
                    a.append(' '.join(splits[int(len(splits)/2):]))
                    
                else:
                    a.append(nst)
    data[s]=a   

In [6]:
with open('/home/kamal/OpenNMT-py/data/scr-body-articles.txt','w') as out:
    for s in data:
        out.write(' '.join(data[s]))
        out.write('\n')


In [36]:
# Returns (3,num_words,1024)

from allennlp.commands.elmo import ElmoEmbedder
from allennlp.modules.elmo import Elmo, batch_to_ids

options_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"
elmo = ElmoEmbedder(options_file, weight_file)


In [34]:
elmo_body = dict()
counter = 0
for art in data:
    embeddings = elmo.embed_sentences(data[art])
    temp = np.zeros(1024)
    for x in data[art]:
        emb = next(embeddings)
        emb = np.mean(np.mean(emb,axis = 0),axis = 0)
        temp = temp+emb
    elmo_body[art] = temp/len(data[art])
    if counter%200 ==0:
        print (counter)
    counter+=1

0


KeyboardInterrupt: 

In [ ]:
np.save('elmo_body.npy',elmo_body)